In [5]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('Player Streak').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 16:44:11 WARN Utils: Your hostname, Pulastyas-Mac-mini.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface en1)
26/01/19 16:44:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 16:44:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/19 16:44:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
players_result = spark.read.options(header=True, inferSchema=True)\
    .csv('data/input/players_result.csv')

players_result.show(truncate=False)

+---------+----------+------------+
|player_id|match_date|match_result|
+---------+----------+------------+
|401      |2021-05-04|W           |
|401      |2021-05-09|L           |
|401      |2021-05-16|L           |
|401      |2021-05-18|W           |
|401      |2021-05-22|L           |
|401      |2021-06-15|L           |
|401      |2021-06-16|W           |
|401      |2021-06-18|W           |
|401      |2021-07-06|L           |
|401      |2021-07-13|L           |
|402      |2021-05-14|L           |
|402      |2021-05-23|L           |
|402      |2021-05-24|W           |
|402      |2021-06-01|W           |
|402      |2021-06-02|W           |
|402      |2021-07-01|W           |
|402      |2021-07-11|W           |
|402      |2021-07-20|L           |
|402      |2021-07-26|L           |
|402      |2021-07-30|L           |
+---------+----------+------------+
only showing top 20 rows


In [21]:
result = players_result \
    .withColumn('is_l', F.when(F.col('match_result') == "L", 1).otherwise(0))\
    .withColumn('island_id', F.sum('is_l').over(Window.partitionBy('player_id').orderBy('match_date')))\
    .filter(F.col('match_result') == 'W')\
    .groupBy(F.col('player_id'), F.col('island_id')).agg(F.count('*').alias('streak'))\
    .select(F.col('player_id'), F.col('streak'))

max_streak = result.groupBy().agg(F.max(F.col('streak'))).collect()[0][0]

player_with_max_streak = result.filter(F.col('streak') == max_streak)

player_with_max_streak.show(truncate=False)

+---------+------+
|player_id|streak|
+---------+------+
|402      |5     |
|403      |5     |
+---------+------+

